In [1]:
from utils import stockutils, basicutils, matchutils, stringparsing, sql_config
import re
try:
    import pandas_datareader.data as web, datetime
    market_data = True
except ImportError:
    print('Market analysis unavailable.')

In [2]:
class StockData:
    def __init__(self, update = False):
        self.company_table = stockutils.company_table(update = update)
    
    @basicutils.memo
    def lookup(self, search_info = ('symbol', 'name', 'se'), num_results = 5, fuzzy_search = True, **conditions):
        """Converts company_name to ticker symbol. Returns a tuple of (symbol, company name) 
        or returns an empty table if company or ticker is not found."""
        initial = conditions.copy()
        if isinstance(search_info, str):
            current_search = [search_info[:]]
        else:
            current_search = list(search_info)
        order = []
        if 'name' in conditions:
            if fuzzy_search:
                fuzzy_names = []
                if not isinstance(conditions['name'], list):
                    conditions['name'] = [conditions['name']] 
                for name in conditions['name']:
                    matches = matchutils.fuzzy_search(name, self.company_table.keys(), num_results = num_results)
                    fuzzy_names.extend([self.company_table[match] 
                                        for match in matches[0] or matches[1] or matches[2]])
                order = conditions['name'] = fuzzy_names
            else:
                order = conditions['name']
            current_search.append('name')
        elif 'symbol' in conditions:
            order = conditions['symbol'] if isinstance(conditions['symbol'], list) else [conditions['symbol']]
            current_search.append('symbol')
        results = stockutils.sql_search(sql_config.sql_path, sql_config.sql_table_name, current_search, **conditions)
        if order:
            sorted_results = [item[:-1] for identifier in order for item in results if identifier == item[-1]]
            return sorted_results or [("No results found matching following conditions: ", str(initial))]
        return results or [("No results found matching following conditions: ", str(initial))]

    def parse_companies(self, sentence):
        """Given a string, returns all parsed companies."""
        res = []
        companies = stringparsing.fuzzy_parse(sentence, self.company_table.keys())
        for parsed, names in companies:
            res.append([parsed, self.lookup(name = names, fuzzy_search = False)])
        return res
    
    @basicutils.memo
    def delta_stock_price(self, ticker, date):
        """Returns opening price of ticker on date and closing price of ticker on the next day."""
        if market_data:
            try:
                prev_day = date + datetime.timedelta(days = -1)
                next_day = date + datetime.timedelta(days = 1)
                day_data = web.DataReader(ticker, 'google', prev_day, next_day).ix
                return (day_data[str(prev_day)[:10]][3],day_data[str(next_day)[:10]][0])
            except:
                try:
                    prev_day = date
                    next_day = date + datetime.timedelta(days = 1)
                    day_data = web.DataReader(ticker, 'yahoo', prev_day, next_day).ix
                    return (day_data[str(prev_day)[:10]][0],day_data[str(next_day)[:10]][0])
                except:
                    return [None,None]
    
    def news_impact(self, date, headline, dump = True):
        """Give a datetime object and headline, stores and returns parsed compaies 
        and their respective stock movements."""
        result = []
        companies = self.parse_companies(headline)
        if len(companies) > 1 or min([len(company[1]) for company in companies] or [2]) > 1:
            print('Multiple or no companies found, data export disabled.')
            dump = False
        for company in companies:
            for res in company[1][:2]:
                open_price, close_price = self.delta_stock_price(res[0], date)
                if not open_price:
                    result.append('No data available for {0} on {1}'.format(str(date)[:10], res[1]))
                else:
                    net_change = (close_price-open_price)/open_price*100
                    print('Company: {0}\tPrevious Day Close: {1}\tNext Day Open: {2}\tNet Change: {3}'.format(res[1],open_price, close_price, net_change))
                    if abs(net_change) > 3:
                        if net_change < 0:
                            result.append('Negative for {}'.format(res[1]))
                        else:
                            result.append('Positive for {}'.format(res[1]))
                    else:
                        result.append('Neutral for {}'.format(res[1]))
                if dump and open_price:
                    stringparsing.connotation_dump('utils/data/json/word_connotations.json', headline, net_change)
        return result
    
    def impact_prediction(self, headline):
        """Given a headline, returns prediction of parsed companies' market movement."""
        res, bigram_list = [], []
        parsed_words = [word.lower() for word in re.sub(r'[^\w\s]','', headline).split()]
        conn_dict = basicutils.import_json('utils/data/json/word_connotations.json')
        try:
            for bigram in stringparsing.list_bigrams(parsed_words):
                bigram_string = ' '.join(bigram)
                value =  basicutils.avg(basicutils.dict_req(bigram_string, conn_dict)) 
                res.append(value)
                if value:
                    bigram_list.extend(bigram)
        except AssertionError:
            print('Bigrams not factored into prediction.')
        for word in parsed_words:
            if not word in bigram_list:
                value = basicutils.avg(basicutils.dict_req(word, conn_dict))
                res.append(value)
        res = [value for value in res if value]
        predicted_impact = basicutils.avg(res)
        companies = self.parse_companies(headline)
        return [(predicted_impact, company[1]) 
                 for parsed_company in companies for company in parsed_company[1]]
        
        

No file found at utils/data/json/cache/lookup.json.
No file found at utils/data/json/cache/delta_stock_price.json.


In [3]:
w = StockData()

In [4]:
w.news_impact(datetime.datetime(2017,1,5),"Macy's to cut more than 10,000 jobs, close 68 stores")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
No file found at utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [5]:
w.news_impact(datetime.datetime(2017,1,5), "Signs of Macy's slowdown appear as number of empty storefronts grows.")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [6]:
w.news_impact(datetime.datetime(2017,1,5), "Macy's Inc ratings on CreditWatch negative")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [7]:
w.news_impact(datetime.datetime(2017,1,5),"Macy's closing 68 stores; job cuts total 10,000; online shopping results in 'declining traffic' for retail giant")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [8]:
w.news_impact(datetime.datetime(2016, 12, 29),"Sprint shares have risen 41% since the election. Catering to Trump's agenda could help the co achieve its own")

Company: Sprint Corporation	Previous Day Close: 8.72	Next Day Open: 8.87	Net Change: 1.7201834862385155
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


['Neutral for Sprint Corporation']

In [9]:
w.news_impact(datetime.datetime(2017, 1, 5,), "Dismal sales slam Macy's")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [10]:
w.news_impact(datetime.datetime(2017, 1, 5), "Macy's to close stores, cut jobs amid weak sales")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [11]:
w.news_impact(datetime.datetime(2017,1,5), "Macy's cut outlooks on poor holiday sales")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [12]:
w.news_impact(datetime.datetime(2016,12,22), "ConAgra's profit beat sends shares to record high")

Company: ConAgra Brands, Inc.	Previous Day Close: 38.0	Next Day Open: 39.22	Net Change: 3.210526315789471
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


['Positive for ConAgra Brands, Inc.']

In [13]:
w.news_impact(datetime.datetime(2016,12,21), "Chipmaker Micron's forecast powered by improving personal computer market")

Company: Micron Technology, Inc.	Previous Day Close: 20.65	Next Day Open: 22.91	Net Change: 10.944309927360782
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


['Positive for Micron Technology, Inc.']

In [14]:
w.news_impact(datetime.datetime(2017,1,5),"Macy's lowers earning forcast amid slow sales")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [15]:
w.news_impact(datetime.datetime(2016,12,20), "FedEx quarterly profit up but misses expectations; stock falls")

Company: FedEx Corporation	Previous Day Close: 197.62	Next Day Open: 194.5	Net Change: -1.5787875721080886
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


['Neutral for FedEx Corporation']

In [16]:
w.news_impact(datetime.datetime(2017,1,3), "Hyundai collaborates with Google Assistant in connecting homes to cars")

Multiple or no companies found, data export disabled.


[]

In [17]:
w.impact_prediction("Intel's earnings fueled by improving industry market")

Data loaded from utils/data/json/word_connotations.json.
by improving [10.944309927360782]
market [10.944309927360782]


[(10.944309927360782, 'Intel Corporation')]

In [18]:
w.news_impact(datetime.datetime(2017,1,5),"Macy's facing heavy losses with lackluster holiday retail season")

Company: Macy's Inc	Previous Day Close: 35.84	Next Day Open: 30.82	Net Change: -14.006696428571436
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


["Negative for Macy's Inc"]

In [19]:
w.impact_prediction('Sears facing dismal holiday retail season.')

Data loaded from utils/data/json/word_connotations.json.
holiday retail [-14.006696428571436]
retail season [-14.006696428571436]
facing [-14.006696428571436]
dismal [-14.006696428571436]


[(-14.006696428571436, 'Sears Canada Inc. '),
 (-14.006696428571436, 'Sears Holdings Corporation'),
 (-14.006696428571436, 'Sears Hometown and Outlet Stores, Inc.')]

In [20]:
w.impact_prediction('NVIDIA grows in improving consumer electronics market')

Data loaded from utils/data/json/word_connotations.json.
grows [-14.006696428571436]
in [-14.006696428571436]
improving [10.944309927360782]
market [10.944309927360782]


[(-1.5311932506053272, 'NVIDIA Corporation')]

In [21]:
w.impact_prediction('Ford faces legal uphill battle with Takata airbag recall')

Data loaded from utils/data/json/word_connotations.json.
with [-14.006696428571436]


[(-14.006696428571436, 'Ford Motor Company')]

In [22]:
w.news_impact(datetime.datetime(2017,1,3), "Ford rolling out hybrid versions of its iconic Mustang and F-150")

Company: Ford Motor Company	Previous Day Close: 12.2	Next Day Open: 12.77	Net Change: 4.672131147540986
Data loaded from utils/data/json/word_connotations.json.
Data dumped at utils/data/json/word_connotations.json.


['Positive for Ford Motor Company']

In [23]:
w.lookup(search_info = ['symbol', 'name', 'ipoyear', 'marketcap'], name = 'Mgmt')[:4]

[('ARES', 'Ares Management L.P.', '2014', '$3.78B'),
 ('WM', 'Waste Management, Inc.', 'n/a', '$31.39B'),
 ('MDLY', 'Medley Management Inc.', '2014', '$285.6M'),
 ('OMAA', 'OM Asset Management plc', '2016', 'n/a')]

In [24]:
w.lookup(ipoyear = list(range(1998,2000)), se = 'amex')[:4]

[('AMKR', 'Amkor Technology, Inc.', 'nasdaq'),
 ('AXTI', 'AXT Inc', 'nasdaq'),
 ('BEBE', 'bebe stores, inc.', 'nasdaq'),
 ('CFNL', 'Cardinal Financial Corporation', 'nasdaq')]